In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from os import listdir
from sklearn.model_selection import train_test_split
print ('The version of TensorFlow is {}'.format(tf.__version__))

The version of TensorFlow is 1.4.0


In [23]:
root_path = './data'

appended_data = []
for file_name in listdir(root_path):
    file_path = root_path + '/' + file_name.encode().decode('utf-8')
    data_from_one_csv = pd.read_csv(file_path, skiprows=1)
    appended_data.append(data_from_one_csv)
    
data = pd.concat(appended_data, axis=0)
data = data.drop(columns = ['ACN', 'Date', 'Local Time Of Day', 'Ceiling', 'Callback', 'Callback.1', 'Unnamed: 96'])
data = data.rename(index=str, columns={"Flight Phase": "Flight Phase1"})

## drop the rows with empty synopsis description
data = data[pd.notnull(data['Synopsis'])]

X = data.drop(columns = 'Result')
Y_raw = pd.DataFrame(data['Result'])

processed_Y = []
for index, row in Y_raw.iterrows():
    #print (index, row['Result'])
    outcome = row['Result']
    if type(outcome) == np.float:
        res = 'unknown'
        processed_Y.append(res)
    elif ';' in outcome:
        res = str(outcome).split(';')[0]
        processed_Y.append(res)
    else:
        res = outcome
        processed_Y.append(res)

Y = pd.DataFrame(processed_Y, columns = ['Result'])

In [24]:
## compress the number of labels to be predicted --> map result to risk level
rate_nine = ['General Declared Emergency', 'General Physical Injury / Incapacitation', 'Flight Crew Inflight Shutdown', 
             'Air Traffic Control Separated Traffic', 'Aircraft Aircraft Damaged']

rate_seven = ['General Evacuated', 'Flight Crew Landed as Precaution', 'Flight Crew Regained Aircraft Control', 
              'Air Traffic Control Issued Advisory / Alert', 'Flight Crew Landed in Emergency Condition',
              'Flight Crew Landed In Emergency Condition']

rate_five = ['General Work Refused', 'Flight Crew Became Reoriented', 'Flight Crew Diverted', 
             'Flight Crew Executed Go Around / Missed Approach', 
             'Flight Crew Overcame Equipment Problem', 'Flight Crew Rejected Takeoff', 'Flight Crew Took Evasive Action', 
             'Air Traffic Control Issued New Clearance']

rate_three = ['General Maintenance Action', 'General Flight Cancelled / Delayed', 
              'General Release Refused / Aircraft Not Accepted', 
              'Flight Crew Overrode Automation', 'Flight Crew FLC Overrode Automation',
              'Flight Crew Exited Penetrated Airspace', 
              'Flight Crew Requested ATC Assistance / Clarification', 'Flight Crew Landed As Precaution',
              'Flight Crew Returned To Clearance', 'Flight Crew Returned To Departure Airport',
              'Aircraft Automation Overrode Flight Crew']

rate_one = ['General Police / Security Involved', 'Flight Crew Returned To Gate', 'Aircraft Equipment Problem Dissipated', 
            'unknown', 'Air Traffic Control Provided Assistance',
            'General None Reported / Taken', 'Flight Crew FLC complied w / Automation / Advisory']

Y_ = []
for i in range(Y.shape[0]):
    if Y['Result'][i] in rate_nine:
        Y_.append(5)
    elif Y['Result'][i] in rate_seven:
        Y_.append(4)
    elif Y['Result'][i] in rate_five:
        Y_.append(3)
    elif Y['Result'][i] in rate_three:
        Y_.append(2)
    elif Y['Result'][i] in rate_one:
        Y_.append(1)
    else:
        print (Y['Result'][i])

outcomes = np.asarray(Y_)
Y_pred = pd.DataFrame(Y_, index = X.index, columns = ['Result'])
unique, counts = np.unique(outcomes, return_counts=True)

In [25]:
data_rev = X.copy(deep=True)
data_rev['Result'] = Y_pred

## Upsampling the minority categories

In [26]:
from sklearn.utils import resample

df_majority_1 = data_rev[data_rev['Result']==1]
df_majority_3 = data_rev[data_rev['Result']==3]
df_minority_2 = data_rev[data_rev['Result']==2]
df_minority_4 = data_rev[data_rev['Result']==4]
df_minority_5 = data_rev[data_rev['Result']==5]

# Upsample minority class
df_minority_2_upsampled = resample(df_minority_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_4_upsampled = resample(df_minority_4, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results
df_minority_5_upsampled = resample(df_minority_5, 
                                 replace=True,     # sample with replacement
                                 n_samples=20000,    # to match majority class
                                 random_state=123) # reproducible results

df_upsampled = pd.concat([df_majority_1, df_majority_3, df_minority_2_upsampled, df_minority_4_upsampled, 
                          df_minority_5_upsampled])

df_upsampled['Result'].value_counts()

X = df_upsampled.drop(columns = 'Result')
Y_pred = df_upsampled['Result']

unique, counts = np.unique(Y_pred, return_counts=True)
print ('After the upsampling, the number of each item is: \n')
print (unique)
print (counts)

After the upsampling, the number of each item is: 

[1 2 3 4 5]
[20985 20000 20848 20000 20000]


## Split the data

In [27]:
from sklearn.model_selection import train_test_split

test_size_ratio = 0.2
random_split_seed = 100
X_train, X_test, Y_train, Y_test = train_test_split(X['Synopsis'], Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)
Y_test.head()

2601    3
740     1
378     4
372     3
1923    4
Name: Result, dtype: int64

## Pipeline: Naive Bayes

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB(alpha = 1, fit_prior=True)),
                    ])

text_clf.fit(X_train, Y_train)
pred_label = text_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, pred_label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.53      0.46      0.49      4206
          2       0.55      0.63      0.59      3937
          3       0.46      0.40      0.43      4174
          4       0.53      0.56      0.54      4032
          5       0.63      0.70      0.66      4018

avg / total       0.54      0.55      0.54     20367



## Model 1: Support Vector Machine with Linear Kernel

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([('vect', CountVectorizer(stop_words = 'english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='epsilon_insensitive', penalty='l2',
                                            alpha=1e-5, random_state=40,
                                            max_iter=10, tol=None)),
                    ])


parameters = {'clf__loss': ['epsilon_insensitive', 'hinge', 'log', 'huber', 'modified_huber', 'perceptron', 
                            'squared_loss', 'squared_epsilon_insensitive', 'squared_hinge'],
              'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3, 1e-4, 1e-5),
              'clf__penalty': ['l1', 'l2', 'elasticnet'],
              'clf__max_iter': (10, 20, 30, 40, 50, 60, 70, 80, 90, 100)
 }

optimal_parameters = {'clf__loss': ['modified_huber'],
              'vect__ngram_range':  [(1, 2)],
              'tfidf__use_idf': [True],
              'clf__alpha': [1e-5],
              'clf__penalty': ['elasticnet'],
              'clf__max_iter': [80],
 }

gs_clf = GridSearchCV(text_clf, optimal_parameters, n_jobs=-1)

gs_clf.fit(X_train, Y_train)
pred_label = gs_clf.predict(X_test)

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 6)]
print(classification_report(Y_test, pred_label, target_names=target_names))

             precision    recall  f1-score   support

          1       0.69      0.50      0.58      4206
          2       0.80      0.91      0.85      3937
          3       0.63      0.55      0.59      4174
          4       0.78      0.91      0.84      4032
          5       0.88      0.98      0.93      4018

avg / total       0.75      0.77      0.75     20367



In [30]:
print ('Accuracy: ', np.sum(np.equal(Y_test, pred_label).astype(int))/20367)
print ('The best set of parameters is \n', gs_clf.best_params_)

Accuracy:  0.765503019590514
The best set of parameters is 
 {'clf__alpha': 1e-05, 'clf__loss': 'modified_huber', 'clf__max_iter': 80, 'clf__penalty': 'elasticnet', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


## Model 2: Processing other categorical data

In [31]:
## change column names
new_col_name = []
for col in X.columns:
    #print(type(col))
    new_col_name.append(col.replace('/ ', '').replace(' ', '_'))
    
X.columns = new_col_name

## output the headers from the csv file
X.keys()

data_type = []
for item_name in X.keys():
    data_type.append(type(X[item_name][0]))

print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('The unique data types across all the items are:', set(data_type))
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

for item_name in X.keys():
    ## find the number of NaN in this item
    no = np.sum(X[item_name].isna().astype(int))
    #print ('The number of {} with value equal to NaN is {}'.format(item_name, no))
    
    ## Replace the missing value with corresponding values
    if no > 0:
        if type(X[item_name][0]) == np.float64:
            X[item_name].fillna(-1, inplace = True)
        else:
            X[item_name].fillna('unknown', inplace = True)
X['Crew_Size'].head()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
The unique data types across all the items are: {<class 'numpy.float64'>, <class 'float'>, <class 'str'>}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


0     2.0
8     2.0
13    2.0
14    2.0
15    2.0
Name: Crew_Size, dtype: float64

In [32]:
## Location
Locale_Reference = tf.feature_column.categorical_column_with_hash_bucket('Locale_Reference', 
                                                                         hash_bucket_size = len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.categorical_column_with_hash_bucket('State_Reference', 
                                                                        hash_bucket_size = len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.categorical_column_with_hash_bucket('Flight_Conditions', 
                                                                hash_bucket_size = len(set(X['State_Reference'])))
Weather_Elements_Visibility = tf.feature_column.categorical_column_with_hash_bucket('Weather_Elements_Visibility', 
                                                            hash_bucket_size = len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.categorical_column_with_hash_bucket('Work_Environment_Factor', 
                                                            hash_bucket_size = len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.categorical_column_with_hash_bucket('Light', hash_bucket_size = 
                                                              len(set(X['Work_Environment_Factor'])))


## Aircraft
ATC_Advisory = tf.feature_column.categorical_column_with_hash_bucket('ATC_Advisory', 
                                                            hash_bucket_size = len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Operator', 
                                                                hash_bucket_size = len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.categorical_column_with_hash_bucket('Make_Model_Name', 
                                                            hash_bucket_size = len(set(X['Make_Model_Name'])))
Crew_Size = tf.feature_column.numeric_column('Crew_Size', [1])
Flight_Plan = tf.feature_column.categorical_column_with_hash_bucket('Flight_Plan', 
                                                            hash_bucket_size = len(set(X['Flight_Plan'])))
Mission = tf.feature_column.categorical_column_with_hash_bucket('Mission', 
                                                                hash_bucket_size = len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.categorical_column_with_hash_bucket('Flight_Phase1', 
                                                                      hash_bucket_size = len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.categorical_column_with_hash_bucket('Route_In_Use', 
                                                                     hash_bucket_size = len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.categorical_column_with_hash_bucket('Airspace', 
                                                                 hash_bucket_size = len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.categorical_column_with_hash_bucket('Aircraft_Component', 
                                                             hash_bucket_size = len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.categorical_column_with_hash_bucket('Manufacturer', 
                                                        hash_bucket_size = len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.categorical_column_with_hash_bucket('Location_Of_Person', 
                                                                hash_bucket_size = len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.categorical_column_with_hash_bucket('Location_In_Aircraft',
                                                            hash_bucket_size = len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.categorical_column_with_hash_bucket('Reporter_Organization',
                                                            hash_bucket_size = len(set(X['Reporter_Organization'])))
Function = tf.feature_column.categorical_column_with_hash_bucket('Function', hash_bucket_size = len(set(X['Function'])))
Qualification = tf.feature_column.categorical_column_with_hash_bucket('Qualification', 
                                                                      hash_bucket_size = len(set(X['Qualification'])))
Human_Factors = tf.feature_column.categorical_column_with_hash_bucket('Human_Factors', 
                                                                      hash_bucket_size = len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.categorical_column_with_hash_bucket('Anomaly', 
                                                                hash_bucket_size = len(set(X['Anomaly'])))
Detector = tf.feature_column.categorical_column_with_hash_bucket('Detector', 
                                                                 hash_bucket_size = len(set(X['Detector'])))
When_Detected = tf.feature_column.categorical_column_with_hash_bucket('When_Detected', 
                                                                      hash_bucket_size = len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.categorical_column_with_hash_bucket('Were_Passengers_Involved_In_Event',
                                                    hash_bucket_size = len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.categorical_column_with_hash_bucket('Contributing_Factors_Situations', 
                                                   hash_bucket_size = len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.categorical_column_with_hash_bucket('Primary_Problem', 
                                                        hash_bucket_size = len(set(X['Primary_Problem'])))

## Place
Locale_Reference = tf.feature_column.embedding_column(Locale_Reference, len(set(X['Locale_Reference'])))
State_Reference = tf.feature_column.embedding_column(State_Reference, len(set(X['State_Reference'])))


## Environment
Flight_Conditions = tf.feature_column.embedding_column(Flight_Conditions,  len(set(X['Flight_Conditions'])))
Weather_Elements_Visibility = tf.feature_column.embedding_column(Weather_Elements_Visibility,  
                                                                 len(set(X['Weather_Elements_Visibility'])))
Work_Environment_Factor = tf.feature_column.embedding_column(Work_Environment_Factor,  len(set(X['Work_Environment_Factor'])))
Light = tf.feature_column.embedding_column(Light, len(set(X['Light'])))


## Aircraft
ATC_Advisory = tf.feature_column.embedding_column(ATC_Advisory, len(set(X['ATC_Advisory'])))
Aircraft_Operator = tf.feature_column.embedding_column(Aircraft_Operator, len(set(X['Aircraft_Operator'])))
Make_Model_Name = tf.feature_column.embedding_column(Make_Model_Name, len(set(X['Make_Model_Name'])))
Flight_Plan = tf.feature_column.embedding_column(Flight_Plan, len(set(X['Flight_Plan'])))
Mission = tf.feature_column.embedding_column(Mission, len(set(X['Mission'])))
Flight_Phase1 = tf.feature_column.embedding_column(Flight_Phase1, len(set(X['Flight_Phase1'])))
Route_In_Use = tf.feature_column.embedding_column(Route_In_Use, len(set(X['Route_In_Use'])))
Airspace = tf.feature_column.embedding_column(Airspace, len(set(X['Airspace'])))

## Component
Aircraft_Component = tf.feature_column.embedding_column(Aircraft_Component, len(set(X['Aircraft_Component'])))
Manufacturer = tf.feature_column.embedding_column(Manufacturer, len(set(X['Manufacturer'])))

## Person
Location_Of_Person = tf.feature_column.embedding_column(Location_Of_Person, len(set(X['Location_Of_Person'])))
Location_In_Aircraft = tf.feature_column.embedding_column(Location_In_Aircraft, len(set(X['Location_In_Aircraft'])))
Reporter_Organization = tf.feature_column.embedding_column(Reporter_Organization, len(set(X['Reporter_Organization'])))
Function = tf.feature_column.embedding_column(Function, len(set(X['Function'])))
Qualification = tf.feature_column.embedding_column(Qualification, len(set(X['Qualification'])))
Human_Factors = tf.feature_column.embedding_column(Human_Factors, len(set(X['Human_Factors'])))

## Events
Anomaly = tf.feature_column.embedding_column(Anomaly, len(set(X['Anomaly'])))
Detector = tf.feature_column.embedding_column(Detector, len(set(X['Detector'])))
When_Detected = tf.feature_column.embedding_column(When_Detected, len(set(X['When_Detected'])))
Were_Passengers_Involved_In_Event = tf.feature_column.embedding_column(Were_Passengers_Involved_In_Event,
                                                                       len(set(X['Were_Passengers_Involved_In_Event'])))

## Assessments
Contributing_Factors_Situations = tf.feature_column.embedding_column(Contributing_Factors_Situations,
                                                                     len(set(X['Contributing_Factors_Situations'])))
Primary_Problem = tf.feature_column.embedding_column(Primary_Problem, len(set(X['Primary_Problem'])))

In [33]:
from sklearn.model_selection import train_test_split

X_sub = X[['Locale_Reference', 'State_Reference', 'Flight_Conditions', 'Weather_Elements_Visibility', 
            'Work_Environment_Factor', 'Light', 'ATC_Advisory', 'Aircraft_Operator', 'Make_Model_Name', 
            'Crew_Size', 'Flight_Plan', 'Mission', 'Flight_Phase1',
            'Route_In_Use','Airspace', 'Aircraft_Component', 'Manufacturer', 'Location_Of_Person', 'Location_In_Aircraft',
            'Reporter_Organization', 'Function', 'Qualification', 'Human_Factors', 'Anomaly', 'Detector', 'When_Detected',
            'Were_Passengers_Involved_In_Event', 'Contributing_Factors_Situations', 'Primary_Problem']]

X_train, X_test, Y_train, Y_test = train_test_split(X_sub, Y_pred, test_size = test_size_ratio, 
                                                    random_state = random_split_seed)

In [34]:
label = []
number_models = 10
for i in range(number_models):
    print ('Train the {} model, please keep waiting !!!'.format(i+1))
    print ('\n')
    
    X_train_set, X_test_tmp, Y_train_set, Y_test_tmp = train_test_split(X_train, Y_train, test_size = 0.2, random_state = i)

    ## define input function
    input_func = tf.estimator.inputs.pandas_input_fn(x = X_train_set, y = Y_train_set, batch_size = 500, 
                                                        num_epochs = 600, shuffle = True)

    ## define the feature columns
    feat_cols = [Locale_Reference, State_Reference, Flight_Conditions, Weather_Elements_Visibility, Work_Environment_Factor, 
                     Light, ATC_Advisory, Aircraft_Operator, Make_Model_Name, Crew_Size, Flight_Plan, Mission, Flight_Phase1, 
                     Route_In_Use, Airspace, Aircraft_Component, Manufacturer, Location_Of_Person, Location_In_Aircraft, 
                     Reporter_Organization, Function, Qualification, Human_Factors, Anomaly, Detector, When_Detected, 
                     Were_Passengers_Involved_In_Event, Contributing_Factors_Situations, Primary_Problem]

    ## build the model
    model = tf.estimator.DNNClassifier(hidden_units = [40, 40, 40, 40, 40, 40, 40, 40], feature_columns = feat_cols,
                                           n_classes = 6, optimizer = tf.train.AdamOptimizer(learning_rate = 0.001))
        
    ## train the model
    model.train(input_fn = input_func, steps = 4000)
    
    
    ## make predictions
    eval_input = tf.estimator.inputs.pandas_input_fn(x = X_test, shuffle = False)
    prediction = list(model.predict(eval_input))

    pred_label = [int(pred['class_ids']) for pred in prediction]
    
    label.append(pred_label)
    
    
ensembel_pred = []
for j in range(len(label[0])):
    x = np.zeros(shape = (len(label), 1)) - 1
    for i in range(len(label)):
        x[i] =  label[i][j]
    (values, counts) = np.unique(x, return_counts=True)
    ind = np.argmax(counts)
    ensembel_pred.append((values[ind]))

ensembel_pred

from sklearn.metrics import classification_report
target_names = [str(i) for i in range(1, 5+1)]
print(classification_report(Y_test, ensembel_pred, target_names=target_names))

Train the 1 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmpra_k3dz9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000207D46D0FD0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpra_k3dz9\model.ckpt.
INFO:tensorflow:loss = 896.07043, step = 1
INFO:tensorflow:global_step/sec: 0.67233
INFO:tensorflow:loss = 568.16705, step = 101 (148.847 sec)
INFO:tensorflow:global_step/sec: 0.67835

INFO:tensorflow:global_step/sec: 0.599432
INFO:tensorflow:loss = 220.00476, step = 901 (166.826 sec)
INFO:tensorflow:global_step/sec: 0.687696
INFO:tensorflow:loss = 206.98425, step = 1001 (145.415 sec)
INFO:tensorflow:global_step/sec: 0.687252
INFO:tensorflow:loss = 156.89923, step = 1101 (145.506 sec)
INFO:tensorflow:global_step/sec: 0.68803
INFO:tensorflow:loss = 206.35995, step = 1201 (145.339 sec)
INFO:tensorflow:Saving checkpoints for 1204 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp_proef6e\model.ckpt.
INFO:tensorflow:global_step/sec: 0.625782
INFO:tensorflow:loss = 167.54797, step = 1301 (159.800 sec)
INFO:tensorflow:global_step/sec: 0.691789
INFO:tensorflow:loss = 123.76759, step = 1401 (144.556 sec)
INFO:tensorflow:global_step/sec: 0.687202
INFO:tensorflow:loss = 152.6106, step = 1501 (145.506 sec)
INFO:tensorflow:global_step/sec: 0.686279
INFO:tensorflow:loss = 152.58377, step = 1601 (145.724 sec)
INFO:tensorflow:Saving checkpoints for 1607 into C:\Users\ZHANGX~1\AppData\Lo

INFO:tensorflow:global_step/sec: 0.682371
INFO:tensorflow:loss = 112.57283, step = 2501 (146.575 sec)
INFO:tensorflow:global_step/sec: 0.680688
INFO:tensorflow:loss = 93.97526, step = 2601 (146.921 sec)
INFO:tensorflow:global_step/sec: 0.682492
INFO:tensorflow:loss = 88.76721, step = 2701 (146.506 sec)
INFO:tensorflow:Saving checkpoints for 2792 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpaiqnxipi\model.ckpt.
INFO:tensorflow:global_step/sec: 0.610293
INFO:tensorflow:loss = 95.13882, step = 2801 (163.832 sec)
INFO:tensorflow:global_step/sec: 0.68239
INFO:tensorflow:loss = 101.28461, step = 2901 (146.566 sec)
INFO:tensorflow:global_step/sec: 0.684287
INFO:tensorflow:loss = 89.43739, step = 3001 (146.142 sec)
INFO:tensorflow:global_step/sec: 0.68252
INFO:tensorflow:loss = 64.02463, step = 3101 (146.519 sec)
INFO:tensorflow:Saving checkpoints for 3191 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpaiqnxipi\model.ckpt.
INFO:tensorflow:global_step/sec: 0.61171
INFO:tensorflow:loss = 71.64536,

INFO:tensorflow:Loss for final step: 83.63826.
INFO:tensorflow:Restoring parameters from C:\Users\ZHANGX~1\AppData\Local\Temp\tmp04ydkbgt\model.ckpt-4000
Train the 5 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmpi7lpeh0_', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002070A8BC048>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpi7lpeh0_\model.ckpt.
INFO:tensorflow:loss = 895.69

INFO:tensorflow:global_step/sec: 0.624418
INFO:tensorflow:loss = 252.274, step = 801 (160.116 sec)
INFO:tensorflow:global_step/sec: 0.683234
INFO:tensorflow:loss = 255.75714, step = 901 (146.395 sec)
INFO:tensorflow:global_step/sec: 0.681729
INFO:tensorflow:loss = 231.64209, step = 1001 (146.686 sec)
INFO:tensorflow:global_step/sec: 0.681873
INFO:tensorflow:loss = 201.70296, step = 1101 (146.640 sec)
INFO:tensorflow:Saving checkpoints for 1200 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp5jlzhujk\model.ckpt.
INFO:tensorflow:global_step/sec: 0.619883
INFO:tensorflow:loss = 141.89714, step = 1201 (161.304 sec)
INFO:tensorflow:global_step/sec: 0.684519
INFO:tensorflow:loss = 178.96104, step = 1301 (146.112 sec)
INFO:tensorflow:global_step/sec: 0.686195
INFO:tensorflow:loss = 152.18277, step = 1401 (145.739 sec)
INFO:tensorflow:global_step/sec: 0.685376
INFO:tensorflow:loss = 144.53009, step = 1501 (145.995 sec)
INFO:tensorflow:Saving checkpoints for 1601 into C:\Users\ZHANGX~1\AppData\Loc

INFO:tensorflow:Saving checkpoints for 2396 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpc6_p5f4s\model.ckpt.
INFO:tensorflow:global_step/sec: 0.617605
INFO:tensorflow:loss = 92.14977, step = 2401 (161.859 sec)
INFO:tensorflow:global_step/sec: 0.681775
INFO:tensorflow:loss = 87.594086, step = 2501 (146.705 sec)
INFO:tensorflow:global_step/sec: 0.682693
INFO:tensorflow:loss = 69.941, step = 2601 (146.474 sec)
INFO:tensorflow:global_step/sec: 0.682865
INFO:tensorflow:loss = 125.086365, step = 2701 (146.465 sec)
INFO:tensorflow:Saving checkpoints for 2796 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpc6_p5f4s\model.ckpt.
INFO:tensorflow:global_step/sec: 0.621087
INFO:tensorflow:loss = 101.56465, step = 2801 (160.960 sec)
INFO:tensorflow:global_step/sec: 0.683823
INFO:tensorflow:loss = 108.26812, step = 2901 (146.265 sec)
INFO:tensorflow:global_step/sec: 0.680498
INFO:tensorflow:loss = 97.76531, step = 3001 (146.955 sec)
INFO:tensorflow:global_step/sec: 0.680661
INFO:tensorflow:loss = 79.94

INFO:tensorflow:global_step/sec: 0.679582
INFO:tensorflow:loss = 92.8549, step = 3901 (147.146 sec)
INFO:tensorflow:Saving checkpoints for 3983 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpshohvgk1\model.ckpt.
INFO:tensorflow:Saving checkpoints for 4000 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmpshohvgk1\model.ckpt.
INFO:tensorflow:Loss for final step: 57.08585.
INFO:tensorflow:Restoring parameters from C:\Users\ZHANGX~1\AppData\Local\Temp\tmpshohvgk1\model.ckpt-4000
Train the 9 model, please keep waiting !!!


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ZHANGX~1\\AppData\\Local\\Temp\\tmpibk1_foj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00

INFO:tensorflow:global_step/sec: 0.679102
INFO:tensorflow:loss = 330.63992, step = 501 (147.285 sec)
INFO:tensorflow:global_step/sec: 0.68311
INFO:tensorflow:loss = 334.51196, step = 601 (146.389 sec)
INFO:tensorflow:global_step/sec: 0.682152
INFO:tensorflow:loss = 302.16132, step = 701 (146.595 sec)
INFO:tensorflow:Saving checkpoints for 800 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp13wcppf3\model.ckpt.
INFO:tensorflow:global_step/sec: 0.619921
INFO:tensorflow:loss = 273.45563, step = 801 (161.279 sec)
INFO:tensorflow:global_step/sec: 0.683304
INFO:tensorflow:loss = 274.2962, step = 901 (146.370 sec)
INFO:tensorflow:global_step/sec: 0.683996
INFO:tensorflow:loss = 219.30148, step = 1001 (146.202 sec)
INFO:tensorflow:global_step/sec: 0.681245
INFO:tensorflow:loss = 226.0775, step = 1101 (146.789 sec)
INFO:tensorflow:Saving checkpoints for 1200 into C:\Users\ZHANGX~1\AppData\Local\Temp\tmp13wcppf3\model.ckpt.
INFO:tensorflow:global_step/sec: 0.62253
INFO:tensorflow:loss = 210.65845, 

## Hybrid model

In [83]:
dict_count = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for i in range(len(ensembel_pred)):
    if ensembel_pred[i] == pred_label[i]:
        dict_count[pred_label[i]] += 1

In [84]:
dict_count

{1: 2492, 2: 3617, 3: 2511, 4: 3744, 5: 3882}

In [85]:
final_pred = []

model_NN = [[0.61, 0.54, 0.57, 4206],
            [0.79, 0.88, 0.83, 3937],
            [0.55, 0.49, 0.52, 4174],
            [0.83, 0.89, 0.86, 4032],
            [0.90, 0.95, 0.93, 4018],
           ]

model_SVM = [[0.69, 0.50, 0.58, 4206],
             [0.80, 0.91, 0.85, 3937],
             [0.63, 0.55, 0.59, 4174],
             [0.78, 0.91, 0.84, 4032],
             [0.88, 0.98, 0.93, 4018],
            ]

for i in range(len(ensembel_pred)):
    if ensembel_pred[i] == pred_label[i]:
        final_pred.append(pred_label[i])
    else:
        prob = np.zeros(shape = 5)
        
        label_ensembel = int(ensembel_pred[i]-1)
        p1 = (model_NN[label_ensembel][3] - dict_count[label_ensembel+1]) * model_NN[label_ensembel][0]/model_NN[label_ensembel][3]
        prob[label_ensembel] = p1
        
        label_SVM = int(pred_label[i]-1)
        p2 = (model_SVM[label_SVM][3] - dict_count[label_SVM+1]) * model_SVM[label_SVM][0]/model_SVM[label_SVM][3]
        prob[label_SVM] = p2
        
        other_labels = list(set(range(1, 6)) - set([label_ensembel + 1, label_SVM + 1]))
        total_recall = 0
        for i in range(len(other_labels)):
            total_recall += (2 - model_NN[other_labels[i]-1][1] - model_SVM[other_labels[i]-1][1])
            
            
        for i in range(len(other_labels)):    
            prob[other_labels[i]-1] = (1 - p1 - p2) * (2 - model_NN[other_labels[i]-1][1] - model_SVM[other_labels[i]-1][1])/total_recall
    
        print (prob)
        final_pred.append(np.argmax(prob) + 1)

[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.53396938 0.1168058  0.25100383 0.05928571 0.03893527]
[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.28118402 0.11169464 0.51060407 0.05928571 0.03723155]
[0.28118402 0.11169464 0.51060407 0.05928571 0.03723155]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.28118402 0.11169464 0.51060407 0.05928571 0.03723155]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.52630683 0.11512962 0.21913033 0.10964726 0.02978596]
[0.28118402 0.11169464 0.51060407 0.05928571 0.03723155]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.56141062 0.12280857 0.219130

[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.56141062 0.12280857 0.21913033 0.05571429 0.04093619]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.53396938 0.1168058  0.25100383 0.05928571 0.03893527]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.24858298 0.11782062 0.53860857 0.05571429 0.03927354]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.53396938 0.1168058  0.251003

[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.53396938 0.1168058  0.25100383 0.05928571 0.03893527]
[0.56141062 0.12280857 0.21913033 0.05571429 0.04093619]
[0.53446622 0.06421133 0.25100383 0.11134713 0.03897149]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.56141062 0.12280857 0.21913033 0.05571429 0.04093619]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.41055924 0.08980983 0.41055924 0.05928571 0.02978596]
[0.55870871 0.06502413 0.21913033 0.11639765 0.04073918]
[0.24858298 0.11061498 0.50566848 0.1053476  0.02978596]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.4118638  0.09009521 0.4118638  0.05571429 0.03046292]
[0.52630683 0.11512962 0.21913033 0.10964726 0.02978596]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.24858298 0.11782062 0.53860857 0.05571429 0.03927354]
[0.24858298 0.11061498 0.505668

[0.52630683 0.11512962 0.21913033 0.10964726 0.02978596]
[0.4245585  0.06421133 0.4245585  0.05571429 0.03095739]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.53396938 0.1168058  0.25100383 0.05928571 0.03893527]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.50349775 0.11014013 0.25100383 0.10489536 0.03046292]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.24858298 0.11061498 0.50566848 0.1053476  0.02978596]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.21861247 0.21913033 0.20820235 0.07287082]
[0.50349775 0.11014013 0.25100383 0.10489536 0.03046292]
[0.53446622 0.06421133 0.25100383 0.11134713 0.03897149]
[0.53446622 0.06421133 0.25100383 0.11134713 0.03897149]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.52630683 0.11512962 0.21913033 0.10964726 0.02978596]
[0.24858298 0.11782062 0.538608

[0.41026538 0.06421133 0.41026538 0.08547195 0.02978596]
[0.24858298 0.11061498 0.50566848 0.1053476  0.02978596]
[0.52630683 0.11512962 0.21913033 0.10964726 0.02978596]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.56141062 0.12280857 0.21913033 0.05571429 0.04093619]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.41026538 0.06421133 0.41026538 0.08547195 0.02978596]
[0.53396938 0.1168058  0.25100383 0.05928571 0.03893527]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.06421133 0.51091095 0.10643978 0.03725392]
[0.28118402 0.10551397 0.48234959 0.1004895  0.03046292]
[0.24858298 0.21893077 0.25100383 0.2085055  0.07297692]
[0.4245585  0.06421133 0.4245585  0.05571429 0.03095739]
[0.56141062 0.12280857 0.21913033 0.05571429 0.04093619]
[0.55870871 0.06502413 0.21913033 0.11639765 0.04073918]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.24858298 0.06502413 0.53572128 0.1116086  0.03906301]
[0.24858298 0.21893077 0.251003

In [86]:
print(classification_report(Y_test, final_pred, target_names=target_names))

             precision    recall  f1-score   support

          1       0.44      0.51      0.48      4206
          2       0.85      0.78      0.81      3937
          3       0.46      0.47      0.47      4174
          4       0.87      0.81      0.84      4032
          5       0.93      0.90      0.92      4018

avg / total       0.71      0.69      0.70     20367



In [72]:
Y_test.head()

2601    3
740     1
378     4
372     3
1923    4
Name: Result, dtype: int64

In [74]:
final_pred

[4,
 3,
 4,
 1,
 4,
 3,
 1,
 4,
 5,
 2,
 2,
 2,
 3,
 1,
 2,
 3,
 5,
 4,
 4,
 1,
 2,
 5,
 4,
 1,
 1,
 4,
 3,
 3,
 3,
 3,
 2,
 5,
 5,
 3,
 2,
 5,
 4,
 4,
 2,
 3,
 3,
 3,
 5,
 3,
 5,
 1,
 2,
 3,
 4,
 1,
 4,
 1,
 5,
 3,
 4,
 5,
 1,
 4,
 2,
 3,
 4,
 5,
 2,
 3,
 4,
 4,
 5,
 4,
 4,
 2,
 5,
 4,
 5,
 3,
 4,
 4,
 2,
 2,
 4,
 1,
 4,
 1,
 1,
 4,
 1,
 4,
 3,
 2,
 2,
 5,
 1,
 5,
 3,
 3,
 2,
 1,
 5,
 5,
 4,
 2,
 5,
 2,
 4,
 1,
 5,
 2,
 1,
 3,
 5,
 1,
 5,
 1,
 5,
 1,
 1,
 2,
 3,
 3,
 1,
 1,
 5,
 5,
 4,
 3,
 2,
 1,
 3,
 1,
 1,
 1,
 3,
 2,
 1,
 3,
 2,
 2,
 1,
 1,
 5,
 3,
 4,
 1,
 3,
 4,
 1,
 1,
 1,
 5,
 4,
 3,
 4,
 1,
 5,
 3,
 1,
 2,
 4,
 4,
 3,
 3,
 1,
 5,
 3,
 1,
 4,
 5,
 2,
 4,
 4,
 3,
 3,
 5,
 4,
 1,
 5,
 3,
 4,
 2,
 5,
 5,
 1,
 5,
 3,
 1,
 5,
 3,
 2,
 5,
 1,
 1,
 1,
 4,
 1,
 3,
 3,
 3,
 5,
 3,
 2,
 5,
 1,
 5,
 1,
 4,
 4,
 5,
 3,
 1,
 3,
 3,
 1,
 1,
 4,
 2,
 1,
 3,
 4,
 2,
 3,
 1,
 5,
 5,
 3,
 1,
 2,
 3,
 1,
 5,
 1,
 2,
 4,
 2,
 4,
 1,
 1,
 4,
 3,
 3,
 2,
 1,
 3,
 4,
 1,
 2,
 3,
 1,
 5,
 5,
 2,
 4,
